## IS2_S2_cryo2ice_search.ipynb
Alek Petty, adapted from an earlier M. Bagnardi script.


Deescription:
- This notebook searches for semi-coincident ICESat-2 and Sentinel-2 data.
- The code finds Sentinel-2 images with ICESat-2 data falling within the image's footprint.
- The code was updated to read in ATl07 data from the new cloud storage using earthdata, see relevant tutorials:
     https://github.com/nsidc/NSIDC-Data-Tutorials/blob/main/notebooks/ICESat-2_Cloud_Access/ATL06-direct-access_rendered.ipynb
     https://github.com/nsidc/NSIDC-Data-Tutorials/blob/main/notebooks/ICESat-2_Cloud_Access/nsidc_daac_uwg_cloud_access_tutorial_rendered.ipynb (No longer hosted!!)
- Include new filtering to only look for images where we have cryo2ice (CryoSat-2 coincident) data based on external file. 
 
To do:
- Should we use CLOUD_COVERAGE_ASSESSMENT or CLOUDY_PIXEL_PERCENTAGE?
- Integrate the GEE or other cloud-based S-2 catalog. What data is on those comapred to the sentinelhub?
- Check all beams iunstead of just the middle beam.
- Add in other imagery/satellite sensors.
- 

Notes:
- Another way of doing this is to use the RGT (reference ground track) data from ICESat-2.
    This could help if you want to look at future data I guess, but downside is it doesn't say where the data actually is on yhe ground, 
    just theoretical tracks.
- Uses the middle strong beam but could be easily adapted to check across all beams.=
- Tested with the adapted 'geo' conda env on cryocloud if you want to convert into a python script.

Output:

Update history:
 - Current notebook created in Apr 2024.


In [1]:
# Uncomment these out when running for the first time on cryocloud
#%pip install sentinelsat
#%pip install earthdata
#%pip install astropy

In [2]:
# Load modules
import datetime as dt
import h5py
import pandas as pd
# Need to do this for geopandas for some reason
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import glob
from shapely.geometry import Point
import matplotlib.pyplot as plt
import os

import numpy as np
from PIL import Image
import io
import requests
import time
import earthaccess
import xarray as xr
from datetime import datetime
from datetime import timedelta

#from astropy.time import Time
#from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt

import ee
import geemap
#Initialize the Google Earth Engine API, handling authentication if necessary.
try:
    ee.Initialize()
except: 
    ee.Authenticate()
    ee.Initialize()

In [3]:
# ICESat-2/earthaccess stuff
auth = earthaccess.login()



Enter your Earthdata Login username:  akpetty
Enter your Earthdata password:  ········


In [4]:
c2ice_df = pd.read_csv('../Data/Cryo2Ice_info_array.csv')
c2ice_df['is2_start_time_dt'] = c2ice_df['start_IS2'].apply(
    lambda x: pd.to_datetime(''.join(x.split(',')), format='%Y %m %d %H %M %S')
)

c2ice_df['is2_end_time_dt'] = c2ice_df['end_IS2'].apply(
    lambda x: pd.to_datetime(''.join(x.split(',')), format='%Y %m %d %H %M %S')
)

c2ice_df['cs2_start_time_dt'] = c2ice_df['start_IS2'].apply(
    lambda x: pd.to_datetime(''.join(x.split(',')), format='%Y %m %d %H %M %S')
)

#reader.vars.avail()
c2ice_files = c2ice_df["atl07filename"].tolist()

In [5]:
#c2ice_df.where(c2ice_df.atl07filename==c2ice_file)
c2ice_df


,l2filename,cpomfilename,atl07filename,atl10filename,start _CS2,end_CS2,start_IS2,end_IS2,strong_beam,is2_start_time_dt,is2_end_time_dt,cs2_start_time_dt
0,CS_LTA__SIR1SAR_FR_20201015T043431_20201015T04...,cry_NO_20201015T043025.elev,ATL07-01_20201015012229_03190901_006_01.h5,ATL10-01_20201015012229_03190901_006_01.h5,"2020, 10, 15, 04, 38, 36","2020, 10, 15, 04, 43, 08","2020, 10, 15, 01, 46, 15","2020, 10, 15, 01, 50, 33",L,2020-10-15 01:46:15,2020-10-15 01:50:33,2020-10-15 01:46:15
1,CS_LTA__SIR1SAR_FR_20210402T024440_20210402T02...,cry_NO_20210402T022221.elev,ATL07-01_20210401235002_01251101_006_02.h5,ATL10-01_20210401235002_01251101_006_02.h5,"2021, 04, 02, 02, 45, 16","2021, 04, 02, 02, 51, 56","2021, 04, 02, 00, 12, 09","2021, 04, 02, 00, 18, 29",R,2021-04-02 00:12:09,2021-04-02 00:18:29,2021-04-02 00:12:09
2,CS_LTA__SIR1SAR_FR_20210403T100237_20210403T10...,cry_NO_20210403T092831.elev,ATL07-01_20210403071550_01451101_006_02.h5,ATL10-01_20210403071550_01451101_006_02.h5,"2021, 04, 03, 10, 06, 52","2021, 04, 03, 10, 13, 43","2021, 04, 03, 07, 34, 07","2021, 04, 03, 07, 40, 37",R,2021-04-03 07:34:07,2021-04-03 07:40:37,2021-04-03 07:34:07
3,CS_LTA__SIR1SAR_FR_20210404T173201_20210404T17...,cry_NO_20210404T173116.elev,ATL07-01_20210404144138_01651101_006_02.h5,ATL10-01_20210404144138_01651101_006_02.h5,"2021, 04, 04, 17, 33, 50","2021, 04, 04, 17, 40, 01","2021, 04, 04, 15, 01, 10","2021, 04, 04, 15, 07, 02",R,2021-04-04 15:01:10,2021-04-04 15:07:02,2021-04-04 15:01:10
4,CS_LTA__SIR1SAR_FR_20210407T082346_20210407T08...,cry_NO_20210407T074556.elev,ATL07-01_20210407053313_02051101_006_02.h5,ATL10-01_20210407053313_02051101_006_02.h5,"2021, 04, 07, 08, 24, 22","2021, 04, 07, 08, 31, 48","2021, 04, 07, 05, 52, 01","2021, 04, 07, 05, 59, 05",R,2021-04-07 05:52:01,2021-04-07 05:59:05,2021-04-07 05:52:01
...,...,...,...,...,...,...,...,...,...,...,...,...
74,CS_OFFL_SIR1SAR_FR_20220329T124106_20220329T12...,cry_NO_20220329T122947.elev,ATL07-01_20220329102930_00971501_006_01.h5,ATL10-01_20220329102930_00971501_006_01.h5,"2022, 03, 29, 12, 41, 48","2022, 03, 29, 12, 46, 23","2022, 03, 29, 10, 49, 35","2022, 03, 29, 10, 53, 57",L,2022-03-29 10:49:35,2022-03-29 10:53:57,2022-03-29 10:49:35
75,CS_OFFL_SIR1SAR_FR_20220329T124106_20220329T12...,cry_NO_20220329T122947.elev,ATL07-01_20220329102930_00971501_006_01.h5,ATL10-01_20220329102930_00971501_006_01.h5,"2022, 03, 29, 12, 46, 22","2022, 03, 29, 12, 50, 06","2022, 03, 29, 10, 53, 56","2022, 03, 29, 10, 57, 30",L,2022-03-29 10:53:56,2022-03-29 10:57:30,2022-03-29 10:53:56
76,CS_OFFL_SIR1SAR_FR_20220401T033307_20220401T03...,cry_NO_20220401T031947.elev,ATL07-01_20220401012104_01371501_006_02.h5,ATL10-01_20220401012104_01371501_006_02.h5,"2022, 04, 01, 03, 34, 01","2022, 04, 01, 03, 41, 31","2022, 04, 01, 01, 42, 02","2022, 04, 01, 01, 49, 10",L,2022-04-01 01:42:02,2022-04-01 01:49:10,2022-04-01 01:42:02
77,CS_OFFL_SIR1SAR_FR_20220402T105853_20220402T11...,cry_NO_20220402T101800.elev,ATL07-01_20220402084652_01571501_006_02.h5,ATL10-01_20220402084652_01571501_006_02.h5,"2022, 04, 02, 10, 59, 29","2022, 04, 02, 11, 04, 47","2022, 04, 02, 09, 07, 39","2022, 04, 02, 09, 12, 43",L,2022-04-02 09:07:39,2022-04-02 09:12:43,2022-04-02 09:07:39


In [ ]:
#find the matched S2 images for each IS2 file
coincident_S2_images=[]

# S2 IDs
coincident_S2_image_ids=[]

#S2 coordinates
S2_coords=[]

#number of S2 images for each IS2 granule
number_of_coincident_scenes_in_granule=[]

# coordinate tracks for each coincident IS2 granule
is2_coordinate_tracks_matched=[] 

# save ATL07 info for the coincident S-2 tracks
ATL_midtimes_matched=[]
ATL_filenames_matched=[]

# Loop over ICESat-2 granules
# Coarsen by this number of data points to speed things up, can play with this more as desired
NPTS=10
# Time difference in minutes
deltatime=180
# Max cloud cover in the S-2 images
max_cloud_cover=100

# filter data close to land (m)
dist_land_mask=10000

image_catalog = "S2_SR"

for c2ice_file in c2ice_files:

    granules_cloud = earthaccess.search_data(
        short_name = 'ATL07',
        version = '006',
        cloud_hosted = True,
        bounding_box = (-180,60,180,90),
        #temporal = ('2020-10-01','2020-10-30'),
        granule_name=c2ice_file,
    )
    
    print(granules_cloud)
    
    # Open files (not found a way of not needing to do this..)
    atl07_file = earthaccess.open(granules_cloud)
    ATL = h5py.File(atl07_file[0], 'r')
    
    orientation = ATL['/orbit_info/sc_orient'][0]

    # Only use central strong beam locations
    if orientation == 0:
        beamID = 'gt2l'
    elif orientation == 1:
        beamID = 'gt2r'
    else:
        print('Spacecraft orientation not found.')

    # Extract data info from granule
    ATL_start_time = ATL['/ancillary_data/data_start_utc'][0]
    ATL_start = pd.to_datetime(ATL_start_time[:-8].decode('utf-8'), format='%Y-%m-%dT%H:%M:%S')
    ATL_end_time = ATL['/ancillary_data/data_end_utc'][0]
    ATL_end = pd.to_datetime(ATL_end_time[:-8].decode('utf-8'), format='%Y-%m-%dT%H:%M:%S')

    ATL_lon = ATL[beamID + '/sea_ice_segments/longitude'][:]
    ATL_lat = ATL[beamID + '/sea_ice_segments/latitude'][:]

    # read in the new dist to land variable to filter these values out now we have a lot more land data in rel006!
    ATL_dist_land = ATL[beamID+'/sea_ice_segments/stats/dist2land'][:]
    ATL_lon = ATL_lon[ATL_dist_land>dist_land_mask]
    ATL_lat = ATL_lat[ATL_dist_land>dist_land_mask]
    atl07_coords = list(zip(ATL_lon[::NPTS], ATL_lat[::NPTS])) 
    
    # Use astropy to convert from gps time to datetime
    #GPS_epoch = ATL['ancillary_data/atlas_sdp_gps_epoch'][:]
    #GPS_time = ATL[beamID + '/sea_ice_segments/delta_time'][::NPTS] + GPS_epoch
    #ATL_tgps = Time(GPS_time, format='gps')
    #ATL_utc = ATL_tgps.utc.iso
    
    
    #create a feature collection representing a line from a list of valid latitude and longitude coordinate points.
    feature_track_temp = ee.FeatureCollection(ee.Geometry.LineString(coords=atl07_coords, proj='EPSG:4326', geodesic=True))
    
    roi = feature_track_temp.geometry()
    
    # Query Sentinel-2 images using ATL07 file information
    #L1C data: COPERNICUS/S2_HARMONIZED S2 (NOT FINDING ANYTHING)
    #L2A data: COPERNICUS/S2_SR_HARMONIZED COPERNICUS/S2_SR (this one is what Wenxuan used)
    
    imagecollection=ee.ImageCollection("COPERNICUS/"+image_catalog).filterBounds(roi)\
    .filterDate(ATL_start- dt.timedelta(minutes=deltatime), ATL_end+ dt.timedelta(minutes=deltatime))\
    .filter(ee.Filter.lt('CLOUD_COVERAGE_ASSESSMENT', max_cloud_cover))
    #.filter(ee.Filter.lt('NODATA_PIXEL_PERCENTAGE', 10))
    
    print('\nCoincident S2 images:', imagecollection.size().getInfo())
    if imagecollection.size().getInfo()>0:
        print('Coincident S2 image FOUND')
        
        # save number of images
        number_of_coincident_scenes_in_granule.append(imagecollection.size().getInfo()) 

        # save actual image collection
        coincident_S2_images.append(imagecollection)

        # S2 image IDs
        S2_ids = [imagecollection.toList(imagecollection.size().getInfo()).getInfo()[i]['id'] for i in range(0, imagecollection.size().getInfo())]
        coincident_S2_image_ids.append(S2_ids)

        # save first coordinte of first image found 
        S2_coord = imagecollection.toList(imagecollection.size().getInfo()).getInfo()[0]['properties']['system:footprint']['coordinates'][0]
        S2_coords.append(S2_coord)
        
        is2_coordinate_tracks_matched.append(feature_track_temp)
        ATL_mid = ATL_start + (ATL_end-ATL_start)/2
        ATL_midtimes_matched.append(ATL_mid)
        ATL_filenames_matched.append(c2ice_file)
        print('Done with granule.')
    
    print('\n\n')





Granules found: 1
[Collection: {'EntryTitle': 'ATLAS/ICESat-2 L3A Sea Ice Height V006'}
Spatial coverage: {'HorizontalSpatialDomain': {'Orbit': {'AscendingCrossing': 48.92460331907133, 'StartLatitude': 27.0, 'StartDirection': 'A', 'EndLatitude': 27.0, 'EndDirection': 'D'}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2020-10-15T01:44:07.383Z', 'EndingDateTime': '2020-10-15T01:50:33.617Z'}}
Size(MB): 181.3192491531372
Data: ['https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL07/006/2020/10/15/ATL07-01_20201015012229_03190901_006_01.h5']]
Opening 1 granules, approx size: 0.18 GB
using provider: NSIDC_CPRD


QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]


Coincident S2 images: 0



Granules found: 1
[Collection: {'EntryTitle': 'ATLAS/ICESat-2 L3A Sea Ice Height V006'}
Spatial coverage: {'HorizontalSpatialDomain': {'Orbit': {'AscendingCrossing': -48.924131734392304, 'StartLatitude': 27.0, 'StartDirection': 'A', 'EndLatitude': 27.0, 'EndDirection': 'D'}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2021-04-02T00:03:38.476Z', 'EndingDateTime': '2021-04-02T00:18:35.195Z'}}
Size(MB): 376.94387912750244
Data: ['https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL07/006/2021/04/02/ATL07-01_20210401235002_01251101_006_02.h5']]
Opening 1 granules, approx size: 0.37 GB
using provider: NSIDC_CPRD


...

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]


Coincident S2 images: 0



Granules found: 1
[Collection: {'EntryTitle': 'ATLAS/ICESat-2 L3A Sea Ice Height V006'}
Spatial coverage: {'HorizontalSpatialDomain': {'Orbit': {'AscendingCrossing': -164.16468328662728, 'StartLatitude': 27.0, 'StartDirection': 'A', 'EndLatitude': 27.0, 'EndDirection': 'D'}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2021-11-15T20:55:22.458Z', 'EndingDateTime': '2021-11-15T21:08:32.301Z'}}
Size(MB): 412.31697177886963
Data: ['https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL07/006/2021/11/15/ATL07-01_20211115203827_08311301_006_01.h5']]
Opening 1 granules, approx size: 0.4 GB
using provider: NSIDC_CPRD


QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Save the output
np.save('../Data/search_output/coincident_S2_image_ids_dt'+str(deltatime)+'_mcc'+str(max_cloud_cover)+'_dlnd'+str(dist_land_mask)+image_catalog, coincident_S2_image_ids)
np.save('../Data/search_output/S2_coords'+str(deltatime)+'_mcc'+str(max_cloud_cover)+'_dlnd'+str(dist_land_mask)+image_catalog, S2_coords)
np.save('../Data/search_output/number_of_coincident_scenes_in_granule'+str(deltatime)+'_mcc'+str(max_cloud_cover)+'_dlnd'+str(dist_land_mask)+image_catalog, number_of_coincident_scenes_in_granule)
np.save('../Data/search_output/is2_coordinate_tracks_matched'+str(deltatime)+'_mcc'+str(max_cloud_cover)+'_dlnd'+str(dist_land_mask)+image_catalog, is2_coordinate_tracks_matched)
np.save('../Data/search_output/ATL_midtimes_matched'+str(deltatime)+'_mcc'+str(max_cloud_cover)+'_dlnd'+str(dist_land_mask)+image_catalog, ATL_midtimes_matched)
np.save('../Data/search_output/ATL_filenames_matched'+str(deltatime)+'_mcc'+str(max_cloud_cover)+'_dlnd'+str(dist_land_mask)+image_catalog, ATL_filenames_matched)

        

In [ ]:
coincident_S2_images

In [ ]:
# MOSAIC images
coincident_S2_mosaics=[]
for imagecollection in coincident_S2_images:
    # mosaic images and save that too
    # more ideas here: https://developers.google.com/earth-engine/apidocs/ee-imagecollection-mosaic
    # could add cloud filters and other things here, for now just creudely mosaic them...
    print('Mosaicing images...')
    mosaic = imagecollection.mosaic()
    coincident_S2_mosaics.append(mosaic)

In [ ]:
Map = geemap.Map()

coincident_granule_num = 1

#coincident_S2_images
#s2_id = coincident_S2_images[coincident_granule_num].aggregate_first('id').getInfo()
mean_cloud_cover = coincident_S2_images[coincident_granule_num].aggregate_mean('CLOUD_COVERAGE_ASSESSMENT').getInfo()
#mean_cloud_pixel = coincident_S2_images[coincident_granule_num].aggregate_mean('CLOUDY_PIXEL_PERCENTAGE').getInfo()
mean_time_delta = coincident_S2_images[coincident_granule_num].aggregate_mean('system:time_start').getInfo()
time_s2 = datetime(1970, 1, 1) + timedelta(seconds = mean_time_delta/1000) 

print(ATL_filenames_matched[coincident_granule_num])
cs2_file_name = c2ice_df[c2ice_df['atl07filename']==ATL_filenames_matched[coincident_granule_num]].l2filename.tolist()[0]
print(cs2_file_name)
print('Coincident S-2 images', coincident_S2_image_ids[coincident_granule_num])

time_cs2_s = c2ice_df[c2ice_df['atl07filename']==ATL_filenames_matched[coincident_granule_num]].cs2_start_time_dt.tolist()[0]
time_is2_s = c2ice_df[c2ice_df['atl07filename']==ATL_filenames_matched[coincident_granule_num]].is2_start_time_dt.tolist()[0]
time_diff_mins_cs2 = np.int(abs(time_cs2_s-time_s2).seconds/60)
time_diff_mins_is2 = np.int(abs(time_is2_s-time_s2).seconds/60)
print(f"Time difference = {time_diff_mins_cs2} minutes between S-2 and start of IS-2 track")
print(f"Time difference = {time_diff_mins_is2_v2} minutes between S-2 and start of CS-2 track")

time_is2_m = ATL_midtimes_matched[coincident_granule_num]
time_diff_mins_is2_mid = np.int(abs(time_is2_m-time_s2).seconds/60)
print(f"Time difference = {time_diff_mins_is2_mid} minutes between S-2 and estimated middle of IS-2 track")

print(f"Cloud cover = {mean_cloud_cover} %")



#print(mean_cloud_cover, time_s2, time_is2, time_diff_mins)

mosaiced_scene = coincident_S2_mosaics[coincident_granule_num]
#img = ee.Image(mosaiced_scene).select('B4','B3','B2')

Map.addLayer(mosaiced_scene, {'bands': ['B4', 'B3', 'B2'], 'max': 10000}, 'RGB Mosaic')

# Center Map around the first image, not figured out how to instead use the mosaic for this
img = ee.Image(coincident_S2_image_ids[coincident_granule_num][0]).select('B4','B3','B2')
Map.centerObject(img, zoom = 7)
Map.addLayer(is2_coordinate_tracks_matched[coincident_granule_num], {}, "ATl07 beam (middle strong)")
#Map.to_html(outfile='./your_map.html', title='My Map', width='100%', height='880px')

# Export the map as a PNG
#url = mosaiced_scene.getThumbURL({
#    'region': geometry.getInfo(),
#    'dimensions': 1024,
#    'format': 'png'
#})

Map

In [ ]:
# Plot a little map showing the location of the S-2 images

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Your list of longitudes and latitudes
# Format: [(lon1, lat1), (lon2, lat2), ...]
coordinates = [(80, 80), (85, 85)]

# Create a figure and add a subplot with a North Polar Stereographic projection
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.NorthPolarStereo())

# Set the extent of the map to focus on the North Pole area
ax.set_extent([-180, 180, 60, 90], ccrs.PlateCarree())

# Add coastlines and borders for context
ax.add_feature(cfeature.COASTLINE)
#ax.add_feature(cfeature.BORDERS, linestyle=':')

# Plot the markers based on the provided coordinates
x=1
for lon, lat in S2_coords:
    ax.plot(lon, lat, 'ro', markersize=5, transform=ccrs.Geodetic())
    ax.text(lon, lat, str(x), transform=ccrs.Geodetic())
    x+=1

plt.savefig('../Data/search_output/locations'+str(deltatime)+'_mcc'+str(max_cloud_cover)+'_dlnd'+str(dist_land_mask)+image_catalog+'.png')
# Display the plot
plt.show()


In [ ]:
#%pip install leafmap

In [ ]:
#import os
#import leafmap.leafmap as leafmap

In [ ]:
#mosaiced_scene

In [ ]:
#m = leafmap.Map()
#m.add("nasa_earth_data")
#m.add_layer(mosaiced_scene,['B4', 'B3', 'B2'])
#url = "https://canada-spot-ortho.s3.amazonaws.com/canada_spot_orthoimages/canada_spot5_orthoimages/S5_2007/S5_11055_6057_20070622/S5_11055_6057_20070622.json"
#leafmap.stac_assets(url)
#leafmap.stac_center(url)
#leafmap.stac_bounds(url)
#m.add_stac_layer(url, bands=["B3", "B2", "B1"])
#m

In [ ]:
#Map.to_html(outfile='./your_map.html', title='My Map', width='100%', height='880px')

In [ ]:
# In this section, we can check single coincident dataset on the map by assigning different values from non_zero_indices to test_x. 
# AP update: I dont need the time check here as it's in my function above!
granule_num=1
scene_number = coincident_granules_idx[granule_num]
S2_ids = [S2_scenes[scene_number].toList(number_of_scenes[scene_number]).getInfo()[i]['id'] for i in range(0, number_of_scenes[scene_number])]  

#time_diff = 3600*4 # Target time difference between ATL07 and Sentinel-2 < 3 hours
#time_IS2 = datetime.strptime(IS2_file_name[test_x][-33:-19], "%Y%m%d%H%M%S")

Map = geemap.Map()

for i in range(0, number_of_scenes[scene_number]):
#for i in range(0, 1):
    time_start = S2_scenes[scene_number].toList(number_of_scenes[scene_number]).getInfo()[i]['properties']['system:time_start']
    time_s2 = datetime(1970, 1, 1) + timedelta(seconds = time_start/1000)   
    
    img_name = os.path.basename(S2_ids[i])

    # Time difference between IS2 and S2 < defined time_diff
    #if abs(time_IS2-time_s2).seconds <= time_diff:

    print(f" {img_name}: Time difference = {np.int(abs(ATL_mid_times[granule_num]-time_s2).seconds/60)} minutes with ICESat-2 track")
    
    img = ee.Image(S2_ids[i]).select('B4','B3','B2')
    dim = img.getInfo()['bands'][0]['dimensions']
    if dim[0] > 10000 and dim[1] > 0:
        # # Download image into the local folder
        #geemap.download_ee_image(img, f"S2_{img_name}.tif", scale=50)
        Map.addLayer(img, {'min': 0, 'max': 10000}, img_name)
        #print(f" {img_name}: Time difference = {abs(time_IS2-time_s2).seconds/3600} hours with ICESat-2 track")
        #else:
        #    print(f"SKIP {img_name}: Not a full image")
   #else:
   #     print(f"SKIP {img_name}: Time difference > {time_diff/3600} hours with ICESat-2 track")
   #     Map.addLayer(img, {'min': 0, 'max': 10000}, img_name)
   #     print(f" {img_name}: Time difference = {abs(time_IS2-time_s2).seconds/3600} hours with ICESat-2 track")

Map.centerObject(img, zoom = 5)
Map.addLayer(feature_track[scene_number], {}, "IS2 track")


Map

In [ ]:
S2[scene_number].toList(number_of_scenes[scene_number]).getInfo()[i]['properties']